# Limpieza y Tokenizacion

- Cargamos los datos 

In [1]:
import pandas as pd
import re
import json

In [9]:
#import spacy
#sp = spacy.load('es_core_news_sm')
#stop_words = sp.Defaults.stop_words
#from nltk.corpus import stopwords
#stopwords_español = stopwords.words('spanish')
#aditionalwords= [palabra for palabra in stopwords_español if palabra not in stop_words ]
#stop_words.extend(aditionalwords)

In [20]:
#Guardar Stop Words
with open('./Data/stopwords.txt', 'w') as f:
    f.write(json.dumps(stop_words))

In [2]:
#Leer Stop words como set
with open('./Data/stopwords.txt', 'r') as f:
    stopwords = json.loads(f.read())
len(stopwords)

704

### Funciones Limpieza de Texto

In [3]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words             containing numbers.'''
    #text = fix_encoding(text)    
    #text = re.sub('[‘’“”…«».]', '', text)
    #text = re.sub('[\n\t]', ' ', text)
    #text = re.sub("(\\d|\\W)+", ' ', text)  # Removing special characters and digits    
    #text = text.replace("_","")

    text = text.lower()
    text = re.sub("\d", '',text)
    pattern = re.compile(r"""                  # Flag para iniciar el modo verbose
              #(?:[A-Za-z]\.|\'|[A-Za-z])+            # Hace match con abreviaciones como U.S.A.Nombre's
              (?:[A-Za-z]\.)+            # Hace match con abreviaciones como U.S.A.        
              | \w+(?:-\w+)*         # Hace match con palabras que pueden tener un guión interno
              # \$?\d+(?:\.\d+)?%?  # Hace match con dinero o porcentajes como $15.5 o 100%
              # \.\.\.              # Hace match con puntos suspensivos
              # [][.,;"'?():-_`]    # Hace match con signos de puntuación
              """, re.X)

    result = pattern.findall(text)
    #nltk.regexp_tokenize(text, pattern)
    return result
    
#text = "En los E.U. U.S.A. esa postal vale $15.50... (feo) `hola` Acoso Escolar (Bullying) en San Juan de Pasto.... J.D.A.A.S. baby Santiago's clubc slsda-sdasd-ss"

#clean_text(text)


In [4]:
resumenes_docentes = pd.read_excel("./Data/Resumenes.xlsx", index_col=0 ,sheet_name = "Proyectos Docentes", dtype="str")
resumenes_estudiantes = pd.read_excel("./Data/Resumenes.xlsx", index_col=0,sheet_name="studiantiles y Trabajos de G", dtype="str")

#Asignamos Columnas
resumenes_docentes.columns = ["codigo", "titulo", "resumen", "estado", "id_autor", "nombre_autor", 
                     "programa", "facultad", "convocatoria", "grupo_investigacion", "linea_investigacion", "palabras_clave"]

resumenes_estudiantes.columns = ['codigo', 'titulo', 'resumen', 'estado', 'id_autor', 'nombre_autor',
       'programa', 'departamento', 'facultad', 'nombre_asesor', 'convocatoria', 'grupo_investigacion', 'linea_investigacion', 'palabras_clave']

#Eliminamos NAN
resumenes_docentes.dropna(inplace=True)
resumenes_estudiantes.dropna(inplace=True)

#Eliminamos registros no encontrados
resumenes_docentes['palabras_clave'] = resumenes_docentes['palabras_clave'].apply(lambda row: row if (row != 'No se encontraron palabras clave registradas') else " ")
resumenes_estudiantes['palabras_clave'] = resumenes_estudiantes['palabras_clave'].apply(lambda row: row if (row != 'No se encontraron palabras clave registradas') else " ")



#### Corpus agregado al DF

In [5]:
#Agregamos Columna de Corpus palabras tokenizadas
resumenes_docentes['corpus'] = resumenes_docentes.apply(lambda row: row['titulo']+' '+row['resumen']+' '+' '+row['palabras_clave']+' '+row['nombre_autor']+' '+row['programa']+' '+' '+row['facultad']+' '+row['grupo_investigacion']+' '+row['linea_investigacion'],axis=1)

resumenes_estudiantes['corpus'] = resumenes_estudiantes.apply(lambda row: row['titulo']+' '+row['resumen']+' '+' '+row['palabras_clave']+' '+row['nombre_autor']+' '+row['nombre_asesor']+' '+row['programa']+' '+row['departamento']+' '+row['facultad']+' '+row['grupo_investigacion']+' '+row['linea_investigacion'],axis=1)

#### Palabras Tokenizadas y Limpias

In [6]:
resumenes_docentes['palabras'] = ''
for index, row in resumenes_docentes.iterrows():
    tokens_corpus = clean_text(row['corpus'])
    row['palabras'] = [palabra for palabra in tokens_corpus if not palabra in stopwords and len(palabra) > 3]

resumenes_estudiantes['palabras'] = ''
for index, row in resumenes_estudiantes.iterrows():
    tokens_corpus = clean_text(row['corpus'])
    row['palabras'] = [palabra for palabra in tokens_corpus if not palabra in stopwords and len(palabra) > 3]

#### Vocabulario Corpus

In [7]:
resumenes_docentes['vocabulario'] = ''
for index, row in resumenes_docentes.iterrows():
    row['vocabulario'] = set(row['palabras'])

resumenes_estudiantes['vocabulario'] = ''
for index, row in resumenes_estudiantes.iterrows():
    row['vocabulario'] = set(row['palabras'])

In [10]:
resumenes_docentes.head(3)

,codigo,titulo,resumen,estado,id_autor,nombre_autor,programa,facultad,convocatoria,grupo_investigacion,linea_investigacion,palabras_clave,corpus,palabras,vocabulario
No.,,,,,,,,,,,,,,,
1.0,1464,"""Diagnóstico del impacto de las iniciativas em...",El emprendimiento es entre otras acepciones es...,Vencido,13515,Francisco Rafael Ayala,Diseño,Artes,Docente 2017,Currículo y Universidad,Curriculos pertinentes,Diagnóstico,"""Diagnóstico del impacto de las iniciativas em...","[diagnóstico, impacto, iniciativas, emprendedo...","{alianzas, egresados, emprendedor, facilitar, ..."
2.0,1367,Acoso Escolar (Bullying) en San Juan de Pasto....,"A nivel mundial, uno de cada tres escolares ha...",Prorroga,11902,Harvey Mauricio Herrera Lopez,Psicología,Ciencias Humanas,Docente 2017,Psicología y Salud,Aspectos Psicosociales en Procesos de Salud,Acoso Escolar,Acoso Escolar (Bullying) en San Juan de Pasto....,"[acoso, escolar, bullying, juan, pasto, modelo...","{agresión, descriptiva, teóricos, mundial, esc..."
3.0,1343,ACTIVIDAD ANTIBACTERIAL DE POLIFENOLES DEL AGU...,El cáncer gástrico (CG) es la principal causa ...,Vencido,10081,Nelson Humberto Hurtado Gutierrez,Química,Ciencias Exactas y Naturales,Docente 2016,GRUPO DE INVESTIGACIÓN EN PRODUCTOS DE IMPORTA...,Modelado molecular,cancer gastrico,ACTIVIDAD ANTIBACTERIAL DE POLIFENOLES DEL AGU...,"[actividad, antibacterial, polifenoles, aguaca...","{medicamentos, preponderante, proceso, mundo, ..."


In [11]:

resumenes_estudiantes.head(3)

,codigo,titulo,resumen,estado,id_autor,nombre_autor,programa,departamento,facultad,nombre_asesor,convocatoria,grupo_investigacion,linea_investigacion,palabras_clave,corpus,palabras,vocabulario
No.,,,,,,,,,,,,,,,,,
1.0,741,"AISLAMIENTO, CARACTERIZACION, ESTABILIDAD Y AC...",El color es una de las cualidades sensoriales ...,Terminado,12324,Omaira Carolina Betancourt Ramos,Química,Química,Ciencias Exactas y Naturales,Nelson Humberto,Adicional de Trabajos de Grado 2013,GRUPO DE INVESTIGACIÓN EN PRODUCTOS DE IMPORTA...,Productos naturales,,"AISLAMIENTO, CARACTERIZACION, ESTABILIDAD Y AC...","[aislamiento, caracterizacion, estabilidad, ac...","{promisoria, permia, enfermedades, muestran, a..."
2.0,760,ANÁLISIS DE ALGORITMOS PARALELOS PARA LA TAREA...,La propuesta de la investigación se centra en ...,Cancelado,12479,Rosa Maria Zambrano Burbano,Ingeniería de Sistemas,Sistemas,Ingeniería,Manuel Ernesto,Adicional de Trabajos de Grado 2013,GRIAS,Deteccione de patrones con tecnicas de mineria...,,ANÁLISIS DE ALGORITMOS PARALELOS PARA LA TAREA...,"[análisis, algoritmos, paralelos, tarea, miner...","{aceptable, implicarían, dcbd, proceso, límite..."
3.0,1412,ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE...,Hoy en día se puede encontrar muchas opciones ...,Vencido,13872,Carlos Mario Coral Cabrera,Ingeniería de Sistemas,Sistemas,Ingeniería,Silvio Ricardo,Estudiantil 2017,GRIAS,Descubrimiento de conocimiento en bases de datos,Análisis de funcionalidad,ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE...,"[análisis, funcionalidad, herramienta, intelig...","{interior, emprender, pymes, facilitar, univer..."


In [18]:
resumenes_docentes.to_csv(r'./Data/ResumenesDocentes.csv')
resumenes_estudiantes.to_csv(r'./Data/ResumenesEstudiantes.csv')


### Lematizar Palabras

In [11]:
#Importar spacy para lemas
import spacy
nlp = spacy.load('es_core_news_sm')

def lemmatizer(text):  
  doc = nlp(text)
  return ' '.join([word.lemma_ for word in doc])



In [22]:
vocabulario = set(resumenes_docentes.loc[1]['palabras'])
vocabulario

{'académicas',
 'académico',
 'académicos',
 'acepciones',
 'actores',
 'actuales',
 'afrontar',
 'alcances',
 'alianzas',
 'aporta',
 'aportes',
 'apoyo',
 'artes',
 'asociado',
 'asociativo',
 'asumir',
 'ayala',
 'beneficio',
 'beneficios',
 'buscando',
 'básica',
 'cambio',
 'cambios',
 'capacidad',
 'capacidades',
 'capaz',
 'caso',
 'casos',
 'catalogan',
 'colectivo',
 'colombiano',
 'comercio',
 'complejas',
 'complejidad',
 'conciencia',
 'condiciones',
 'conjunto',
 'consolidación',
 'contexto',
 'continuo',
 'contribuir',
 'conveniente',
 'creación',
 'crear',
 'creativo',
 'crecimiento',
 'cultura',
 'curriculos',
 'currículo',
 'derecho',
 'desarrollar',
 'desarrollo',
 'desarrollos',
 'describe',
 'descubrir',
 'diagnóstico',
 'dinámica',
 'dinámico',
 'diseño',
 'dispuesto',
 'económicos',
 'ecosistema',
 'egresados',
 'ejercer',
 'emprendedor',
 'emprendedoras',
 'emprendimiento',
 'empresas',
 'enfrentado',
 'entidades',
 'entornos',
 'equipo',
 'escuchar',
 'establece

In [23]:
lemas = [ lemmatizer(palabra) for palabra in vocabulario]

In [27]:
lemas = list(lemas)